- ### import BeautifulSoup , html5lib and requests
they are needed to perform the webscraping 
- ### set the headers which is the browser user agent and the url for the website you want to scrape from
The User-Agent (UA) string is contained in the HTTP headers and is intended to identify devices requesting online content.

In [44]:
import pandas as pd
import requests
import html5lib
from bs4 import BeautifulSoup
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'}
url='https://www.transfermarkt.com/transfers/transfertagedetail/statistik/top/land_id_ab//land_id_zu//leihe//datum/2022-09-12/plus/1/galerie/0/page/1'

Now we will get the html content of the web page 

In [45]:
r=requests.get(url,headers=headers)
soup=BeautifulSoup(r.content,'html5lib')


![](../../Kaggle/football%20market%20transfer/table.png)

This is the needed table that we want to scrape.Using find method we can extract this table out of the whole page by determining the class of the table

In [46]:
table=soup.find('table',attrs={'class':'items'})
print(table)


<table class="items">
<thead>
<tr>
<th class="" id="yw1_c0">player</th><th class="zentriert" id="yw1_c1">Age</th><th class="zentriert" id="yw1_c2">Nat.</th><th class="" id="yw1_c3">Left</th><th class="" id="yw1_c4">Joined</th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/transfers/transfertagedetail/statistik/top/land_id_ab//land_id_zu//leihe//datum/2022-09-12/plus/1/galerie/0/page/1/sort/marktwert_beim_wechsel.desc">Market value at time of transfer</a></th><th class="rechts" id="yw1_c6"><a class="sort-link desc" href="/transfers/transfertagedetail/statistik/top/land_id_ab//land_id_zu//leihe//datum/2022-09-12/plus/1/galerie/0/page/1/sort/abloese">Fee</a></th></tr>
</thead>
<tbody>
<tr class="odd">
<td class="">    <table class="inline-table">
        <tbody><tr>
            <td rowspan="2">
                <img alt="Marwan Hamdi" class="bilderrahmen-fixed lazy lazy" data-src="https://img.a.transfermarkt.technology/portrait/medium/557938-1567955310.jpg?lm=1" src="data:image

### Players Names 

In [47]:
player=[]
players=table.find_all('img',attrs={'class':"bilderrahmen-fixed lazy lazy"})
for row in players:
     player.append(str(str(row).split('" class',1)[0].split('<img alt="',1)[1]))
player

['Marwan Hamdi',
 'Aykut Akgün',
 'Rajko Brezancic',
 'Billel Omrani',
 'Subhasish Roy Chowdhury',
 'Kevin Luckassen',
 'Marlon',
 'Slimen Kchouk',
 'Liam Grimshaw',
 'Pablo Aguilera',
 'Gabriele Aprile',
 'Dion Cools',
 'Kachi',
 'Artiom Carastoian',
 'Abdel Diarra',
 'Marvin Márquez',
 'Thomas Alberti',
 'Adnane Brahni',
 'Marco Di Rienzo',
 'Slavko Radovanovic',
 'Nahum Melvin-Lambert',
 'Alaeddine Ben Salah',
 'Ali Mollaei',
 'Ibrahim Mandefu',
 'Udoka Chima']

### Players Ages and Nationalities
some players have more than one nationality

In [48]:
ages=table.find_all('td',attrs={'class':'zentriert'})
nationality=[]
age=[]
#print(ages)
for i in range(0,len(ages),2):
       age.append(str(ages[i]).split('>',1)[1].split('<',1)[0])
       nat=ages[i+1].find_all('img',attrs={'class':'flaggenrahmen'})
       temp=[]
       for j in nat:
            temp.append(str(j).split('alt="',1)[1].split('" class',1)[0])
       nationality.append(temp)
    

print(age)
print(nationality)

['25', '34', '33', '29', '35', '29', '36', '28', '27', '28', '27', '26', '25', '22', '27', '24', '24', '28', '20', '21', '19', '25', '24', '21', '20']
[['Egypt'], ['Turkey', 'Germany'], ['Serbia', 'Bosnia-Herzegovina'], ['Algeria', 'France'], ['India'], ['Netherlands', 'Ghana'], ['Brazil'], ['Tunisia'], ['England'], ['Spain'], ['Italy'], ['Malaysia', 'Belgium'], ['Nigeria'], ['Moldova', 'Romania'], ["Cote d'Ivoire"], ['El Salvador'], ['Italy'], ['Morocco'], ['Italy'], ['Serbia'], ['England', 'St. Lucia'], ['Tunisia'], ['Iran'], ['Senegal', 'France'], ['England', 'Nigeria']]


### Players Positions

In [49]:
position=[]
pos=[i.find_all('td')[-1] for i in table.find_all('table',attrs={'class':'inline-table'})]
print(len(pos))
for i in range(0,len(pos),3):
   position.append(str(pos[i]).split('>',1)[1].split('<',1)[0])
print(position)

75
['Centre-Forward', 'Defensive Midfield', 'Left-Back', 'Centre-Forward', 'Goalkeeper', 'Centre-Forward', 'Left-Back', 'Centre-Back', 'Right-Back', 'Centre-Forward', 'Defensive Midfield', 'Right-Back', 'Centre-Forward', 'Left Winger', 'Centre-Back', 'Centre-Forward', 'Centre-Forward', 'Centre-Back', 'Goalkeeper', 'Right-Back', 'Centre-Forward', 'Central Midfield', 'Centre-Back', 'Centre-Forward', 'Centre-Back']


### Clubs Information
- It was little bit harder as some playes weren't assigned by a club (there contracts ended with their clubs ) so the data wasn't well organized 
- I got the club transfered from and the country he was playing at also the club transfered to and its country 
- also got the leagues of the clubs but sometimes its missing 

In [50]:
from attr import attr


from_club=[]
from_league=[]
from_country=[]

to_club=[]
to_league=[]
to_country=[]
#the inline table contains the data of the clubs 
info=table.find_all('table',attrs={'class':'inline-table'})

from_info=[]
to_info=[]
# the info is divided into 3 inline tables the first for the players and we will ignore it the second is the from_info and the third is the to_info
for i in range(1,len(info),3):
        #the from Data is divided into 2 tr one for the club and the other for the league and the country 
        for j in info[i].find_all('tr'):
                z=j.find_all('td')[0]
                if z.find_all('a')==[]  :
                        from_info.append('unknown_league')
                        continue
                from_info.append(str(z).split('href="/',1)[1].split('/',1)[0])
        if z.find_all('img')==[] :
                from_country.append("unknown_country")
        else:
                from_country.append(str(z.find_all('img',attrs={'class':'flaggenrahmen'})[0]).split('alt="',1)[1].split('"',1)[0])

        #the to Data is divided into 2 tr one for the club and the other for the league and the country 

        for j in info[i+1].find_all('tr'):
                z=j.find_all('td')[0]
                if z.find_all('a')==[] :
                        to_info.append('unknown_league')
                        continue
                to_info.append(str(z).split('href="/',1)[1].split('/',1)[0])
        if z.find_all('img')==[] :
                to_country.append("unknown_country")
        else:
                to_country.append(str(z.find('img',attrs={'class':'flaggenrahmen'})).split('alt="',1)[1].split('"',1)[0])

#arrange everything 

for i in range(0,len(from_info),2):
        from_club.append(from_info[i])
        from_league.append(from_info[i+1])
for i in range(0,len(to_info),2):
        to_club.append(to_info[i])
        to_league.append(to_info[i+1])


### The Market value of the player and the transfer fee
The market value is the price estimated of the player while the transfer fee is the real cost of the transfer

In [51]:
market_value=[]
fee=[]

money=table.find_all('td',attrs={'class':["rechts"]})
for i in range(0,len(money),2):
       market_value.append(str(money[i]).split('>',1)[1].split('<',1)[0])
       fee.append(str(money[i+1]).split('>',2)[2].split('<',1)[0])
print(market_value)
print(fee)

['€500Th.', '€25Th.', '€100Th.', '€1.60m', '€50Th.', '€800Th.', '€150Th.', '€100Th.', '€325Th.', '€50Th.', '€50Th.', '€850Th.', '€100Th.', '€150Th.', '€125Th.', '€75Th.', '€100Th.', '€150Th.', '€100Th.', '€25Th.', '€75Th.', '€50Th.', '€200Th.', '€100Th.', '€25Th.']
['€1.00m', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer', 'loan transfer', 'free transfer', 'free transfer', 'free transfer', 'free transfer']


In [52]:
print(len(player),len(position),len(age),len(nationality),len(from_club),len(to_club),len(to_country),len(from_country),len(market_value),len(fee),len(from_league),len(to_league))

25 25 25 25 25 25 25 25 25 25 25 25


### Create the DataFrame Using Pandas

In [53]:

data={'name':player,'age':age,'nationality':nationality,'from_club':from_club,'from_league':from_league,'from_country':from_country,'to_club':to_club,'to_league':to_league,'to_country':to_country,'market_value':market_value,'transfer_value':fee}
df=pd.DataFrame(data)
df.head(25)

,name,age,nationality,from_club,from_league,from_country,to_club,to_league,to_country,market_value,transfer_value
0,Marwan Hamdi,25,[Egypt],zamalek-sc,egyptian-premier-league,Egypt,el-masry-sc,egyptian-premier-league,Egypt,€500Th.,€1.00m
1,Aykut Akgün,34,"[Turkey, Germany]",corum-fk,2-lig-beyaz,Turkey,gemlik-vefa-spor,unknown_league,Turkey,€25Th.,free transfer
2,Rajko Brezancic,33,"[Serbia, Bosnia-Herzegovina]",fk-partizan-belgrad,super-liga-srbije,Serbia,fk-radnicki-belgrad,prva-liga-srbije,Serbia,€100Th.,free transfer
3,Billel Omrani,29,"[Algeria, France]",cfr-cluj,liga-1,Romania,fcsb,liga-1,Romania,€1.60m,free transfer
4,Subhasish Roy Chowdhury,35,[India],northeast-united-fc,indian-super-league,India,real-kashmir-fc,i-league,India,€50Th.,free transfer
5,Kevin Luckassen,29,"[Netherlands, Ghana]",kayserispor,super-lig,Turkey,fc-rapid-1923,liga-1,Romania,€800Th.,free transfer
6,Marlon,36,[Brazil],clube-do-remo-pa-,campeonato-brasileiro-serie-c,Brazil,unbekannt,unknown_league,unknown_country,€150Th.,free transfer
7,Slimen Kchouk,28,[Tunisia],vereinslos,unknown_league,unknown_country,al-khums-sc,unknown_league,Libya,€100Th.,free transfer
8,Liam Grimshaw,27,[England],vereinslos,unknown_league,unknown_country,greenock-morton-fc,scottish-championship,Scotland,€325Th.,free transfer
9,Pablo Aguilera,28,[Spain],racing-murcia-fc,unknown_league,Spain,ud-torre-del-mar,unknown_league,Spain,€50Th.,free transfer
